
## 1. Importar Paquetes y Datos

In [95]:
# Operating System
import os

# Packages for strings
import re
from unidecode import unidecode

# Packages for data analysis

import numpy as np
import pandas as pd
from scipy import stats
from itertools import combinations
import pingouin as pg

# Custom Preprocess
from preprocess_module import binning_cols

# Packages for visualizations
import matplotlib.pyplot as plt
import seaborn as sns

In [96]:
df0 = pd.read_csv('ResultadoClusteringBogota.csv')
df0 = df0.drop('Unnamed: 0', axis=1)
#,index_col=1

In [97]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537 entries, 0 to 536
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   cod_sect                      537 non-null    int64  
 1   conteo_sucursales             537 non-null    float64
 2   conteo_cbs_uno_a_uno          537 non-null    float64
 3   conteo_cbs_robustos           537 non-null    float64
 4   conteo_cbs_agregador          537 non-null    float64
 5   conteo_atms_dispensadores     537 non-null    float64
 6   conteo_atms_multifuncionales  537 non-null    float64
 7   Cluster                       537 non-null    object 
dtypes: float64(6), int64(1), object(1)
memory usage: 33.7+ KB



###  Lectura y filtros

In [98]:
# Leer el csv
df = pd.read_csv('Base_Nueva_Microsegmentacion_Indicadores.csv', delimiter=';')

# Función para limpiar los nombres
def clean_string(string):
    # Remove spaces
    string = re.sub(r'\s', '_', string)
    # Remove accents
    string = unidecode(string)
    # Remove punctuation and replace hyphens with underscores
    string = re.sub(r'[^\w\s-]', '', string)
    string = re.sub(r'-', '_', string)
    # Convert to lowercase
    string = string.lower()
    return string

# Transformar el nombre de las columnas
df.columns = list(map(clean_string, list(df.columns)))

# Filtrar los datos para obtener unicamente la información de Bogotá
df = df.loc[df['cod_cpob'] == 11001000].copy().reset_index(drop=True)

In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537 entries, 0 to 536
Data columns (total 86 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   cod_dpto                                                      537 non-null    int64  
 1   cod_mpio                                                      537 non-null    int64  
 2   cod_cpob                                                      537 non-null    int64  
 3   cod_sect                                                      537 non-null    int64  
 4   shape_leng                                                    537 non-null    float64
 5   shape_area                                                    537 non-null    float64
 6   setu_ccdgo                                                    537 non-null    int64  
 7   geometry                                                      537 non-n

In [113]:
prueba = df0.merge(df.iloc[:, [3,8,9,10,11,39,67,68,71,72,75,76,79,80,83,84]], how='left', left_on="cod_sect",right_on="cod_sect")

## 2. Desbalanceo y reemplazo de outliers

In [101]:
# Highly imbalance categorical columns and high zeros columns -> BINNING
categorical_imbalance = [
    'conteo_sucursales',
    'conteo_cbs_robustos',
    'conteo_atms_multifuncionales',
]

for col in categorical_imbalance:
    list_val =  prueba[col].unique().tolist()
    list_val.sort()
    print(col, list_val)

print('-'*60)

prueba = binning_cols(prueba, categorical_imbalance, threshold_percentage=0.1)

for col in categorical_imbalance:
    list_val =  prueba[col].unique().tolist()
    list_val.sort()
    print(col, list_val)

conteo_sucursales [0.0, 1.0, 2.0]
conteo_cbs_robustos [0.0, 1.0, 2.0, 3.0]
conteo_atms_multifuncionales [0.0, 1.0, 2.0, 3.0, 4.0]
------------------------------------------------------------
conteo_sucursales [0.0, 1.0]
conteo_cbs_robustos [0.0, 1.0]
conteo_atms_multifuncionales [0.0, 1.0]


In [114]:
prueba.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 537 entries, 0 to 536
Data columns (total 23 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   cod_sect                                      537 non-null    int64  
 1   conteo_sucursales                             537 non-null    float64
 2   conteo_cbs_uno_a_uno                          537 non-null    float64
 3   conteo_cbs_robustos                           537 non-null    float64
 4   conteo_cbs_agregador                          537 non-null    float64
 5   conteo_atms_dispensadores                     537 non-null    float64
 6   conteo_atms_multifuncionales                  537 non-null    float64
 7   Cluster                                       537 non-null    object 
 8   conteo_clientes                               537 non-null    float64
 9   promedio_ingresos_mensuales                   537 non-null    flo

In [103]:
def replace_outliers_whisker(df, columns):
    copy_df = df.copy()
    for column in columns:
        Q1 = copy_df[column].quantile(0.25)
        Q3 = copy_df[column].quantile(0.75)
        IQR = Q3 - Q1
        whisker_width = 1.5
        lower_whisker = Q1 - (whisker_width*IQR)
        upper_whisker = Q3 + (whisker_width*IQR)
        copy_df.loc[copy_df[column] < lower_whisker, column] = lower_whisker
        copy_df.loc[copy_df[column] > upper_whisker, column] = upper_whisker
    return copy_df

continuous_columns = list(set(prueba.iloc[:, 8:].columns).difference(set(categorical_imbalance)))
prueba = replace_outliers_whisker(prueba, continuous_columns)

In [104]:
prueba['Cluster'].value_counts()

05 UNICANAL: SOLO 1A1                   123
07 BICANAL: AGR & 1A1                   107
08 TRICANAL: ATM & AGR & 1A1             55
06 BICANAL: ATM & 1A1                    53
00 MULTICANAL: Sucursal                  46
01 MULTICANAL: CB Robusto                38
02 MULTICANAL: CB Robusto + Sucursal     38
04 UNICANAL: SOLO AGR                    30
03 UNICANAL: SOLO ATM                    28
99 De todo un poco (Noise)               19
Name: Cluster, dtype: int64

In [12]:
# Creación de los diferentes subsets por cluster
#dc0 = prueba[prueba['Cluster'] == '00 MULTICANAL: Sucursal']
#dc1 = prueba[prueba['Cluster'] == '01 MULTICANAL: CB Robusto']
#dc2 = prueba[prueba['Cluster'] == '02 MULTICANAL: CB Robusto + Sucursal']
#dc3 = prueba[prueba['Cluster'] == '03 UNICANAL: SOLO ATM']
#dc4 = prueba[prueba['Cluster'] == '04 UNICANAL: SOLO AGR']
#dc5 = prueba[prueba['Cluster'] == '05 UNICANAL: SOLO 1A1']
#dc6 = prueba[prueba['Cluster'] == '06 BICANAL: ATM & 1A1']
#dc7 = prueba[prueba['Cluster'] == '07 BICANAL: AGR & 1A1']
#dc8 = prueba[prueba['Cluster'] == '08 TRICANAL: ATM & AGR & 1A1']
#dc9 = prueba[prueba['Cluster'] == '99 De todo un poco (Noise)']

In [115]:
# Sucursales
df_suc = prueba[prueba['Cluster'].isin(['00 MULTICANAL: Sucursal',
                                         '02 MULTICANAL: CB Robusto + Sucursal'])]

#ATMS DIS
df_atms = prueba[prueba['Cluster'].isin(['00 MULTICANAL: Sucursal',
                                         '01 MULTICANAL: CB Robusto',
                                         '02 MULTICANAL: CB Robusto + Sucursal',
                                         '03 UNICANAL: SOLO ATM',
                                         '06 BICANAL: ATM & 1A1',
                                         '08 TRICANAL: ATM & AGR & 1A1'])]

#CBS Uno A Uno y Agregadores "informales"  
df_cbs_1 = prueba[prueba['Cluster'].isin(['04 UNICANAL: SOLO AGR',
                                          '05 UNICANAL: SOLO 1A1',
                                          '06 BICANAL: ATM & 1A1',
                                          '07 BICANAL: AGR & 1A1',
                                          '08 TRICANAL: ATM & AGR & 1A1'])]

#CBS robusto "formales" 
df_cbs_2 = prueba[prueba['Cluster'].isin(['01 MULTICANAL: CB Robusto',
                                          '02 MULTICANAL: CB Robusto + Sucursal'])]


## 3. Analisis

### ATMs Dispensadores

In [121]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import MultiComparison
modelo_anova = ols('conteo_transacciones_atms_dispensadores ~ Cluster', data=df_atms).fit()
anova_tabla = sm.stats.anova_lm(modelo_anova, typ=2)

# Crea un objeto MultiComparison para el análisis de Tukey
mc = MultiComparison(df_atms['conteo_transacciones_atms_dispensadores'], df_atms['Cluster'])

# Realiza el análisis de Tukey
resultado_tukey = mc.tukeyhsd()

# Imprime el resultado del análisis de Tukey
print(resultado_tukey)

                                    Multiple Comparison of Means - Tukey HSD, FWER=0.05                                    
               group1                               group2                  meandiff  p-adj     lower       upper    reject
---------------------------------------------------------------------------------------------------------------------------
             00 MULTICANAL: Sucursal            01 MULTICANAL: CB Robusto  -7848.4851 0.4049 -19715.0322    4018.062  False
             00 MULTICANAL: Sucursal 02 MULTICANAL: CB Robusto + Sucursal  38192.5412    0.0  26325.9941  50059.0883   True
             00 MULTICANAL: Sucursal                03 UNICANAL: SOLO ATM -17476.2314 0.0019 -30451.4106  -4501.0521   True
             00 MULTICANAL: Sucursal                06 BICANAL: ATM & 1A1 -17157.6202 0.0001 -28065.8979  -6249.3425   True
             00 MULTICANAL: Sucursal         08 TRICANAL: ATM & AGR & 1A1 -11753.2775 0.0244 -22569.0082   -937.5468   True
        

In [107]:
df_atms.groupby('Cluster')['conteo_atms_dispensadores'].sum()

Cluster
00 MULTICANAL: Sucursal                 216.0
01 MULTICANAL: CB Robusto                94.0
02 MULTICANAL: CB Robusto + Sucursal    337.0
03 UNICANAL: SOLO ATM                    50.0
06 BICANAL: ATM & 1A1                   100.0
08 TRICANAL: ATM & AGR & 1A1            120.0
Name: conteo_atms_dispensadores, dtype: float64

In [108]:
df_atms.groupby('Cluster')['conteo_transacciones_atms_dispensadores'].sum()

Cluster
00 MULTICANAL: Sucursal                 1315420.0
01 MULTICANAL: CB Robusto                793421.0
02 MULTICANAL: CB Robusto + Sucursal    1770311.0
03 UNICANAL: SOLO ATM                    360149.0
06 BICANAL: ATM & 1A1                    698597.0
08 TRICANAL: ATM & AGR & 1A1            1005993.0
Name: conteo_transacciones_atms_dispensadores, dtype: float64

In [109]:
df_atms.groupby('Cluster')['conteo_transacciones_atms_dispensadores'].mean()

Cluster
00 MULTICANAL: Sucursal                 28596.086957
01 MULTICANAL: CB Robusto               20879.500000
02 MULTICANAL: CB Robusto + Sucursal    46587.131579
03 UNICANAL: SOLO ATM                   12862.464286
06 BICANAL: ATM & 1A1                   13181.075472
08 TRICANAL: ATM & AGR & 1A1            18290.781818
Name: conteo_transacciones_atms_dispensadores, dtype: float64

In [110]:
df_atms.groupby('Cluster')['conteo_transacciones_atms_dispensadores'].median()

Cluster
00 MULTICANAL: Sucursal                 28761.0
01 MULTICANAL: CB Robusto               18369.5
02 MULTICANAL: CB Robusto + Sucursal    52255.0
03 UNICANAL: SOLO ATM                   10424.5
06 BICANAL: ATM & 1A1                   10288.0
08 TRICANAL: ATM & AGR & 1A1            15395.0
Name: conteo_transacciones_atms_dispensadores, dtype: float64

**Conclusiones:** 

1. Al comparar los cluster 2-3, 2-6 y 2-8 podemos evidenciar que hay diferencias significativas en la cantidad promedio de transacciones en atms dispensadores, es claro que en el cluster dos predomina el numero medio de transacciones sobre los demas cluster debido a que es en el que mas cantidad de atms encontramos, sin embargo, este no es el canal que sobresale en el criterio de clasificación. 


2. Si tenemos en cuenta los cluster 3,6,7 en los cuales los atms tienen un comportamiento similar en su numero promedio de transacciones segun la prueba, cabe recalcar que si los juntamos **(tanto en cantidad de atms como en numero promedio de transacciones)** no se alcanza a superar el numero que aporta el cluster 2. De acuerdo con esto, podriamos decir que el hecho de que los atms esten complementados a sus alrededores con CB robustos y sucursales ayuda a que haya una mayor cantidad de transacciones en los atms dispensadores. La anterior conclusion tiene sentido debido a que los cb Robustos son el formato que se podria ver como mas formal en comparacion con los demas y la sucursal es aquel espacio donde el cliente se siente mas cercano al banco. Notemos que la anterior hipotesis al analizar con respecto a los otros dos cluster que son el 0 y el 1 tambien observamos altos valores de cantidades promedio de transacciones y estos canales estan respaldados individualmente en el cluster 00 por sucursales y en el cluster 1 por CB robustos.

### Corresponsales Agregador

In [112]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import MultiComparison
#promedio_transacciones_atms_multifuncionales
modelo_anova = ols('conteo_transacciones_cbs_agregador ~ Cluster', data=df_cbs_1).fit()
anova_tabla = sm.stats.anova_lm(modelo_anova, typ=2)

# Crea un objeto MultiComparison para el análisis de Tukey
mc = MultiComparison(df_cbs_1['conteo_transacciones_cbs_agregador'], df_cbs_1['Cluster'])

# Realiza el análisis de Tukey
resultado_tukey = mc.tukeyhsd()

# Imprime el resultado del análisis de Tukey
print(resultado_tukey)

                      Multiple Comparison of Means - Tukey HSD, FWER=0.05                       
        group1                   group2             meandiff  p-adj    lower      upper   reject
------------------------------------------------------------------------------------------------
04 UNICANAL: SOLO AGR        05 UNICANAL: SOLO 1A1 -1152.7099    0.0 -1629.2781 -676.1418   True
04 UNICANAL: SOLO AGR        06 BICANAL: ATM & 1A1 -1152.7099    0.0  -1687.438 -617.9819   True
04 UNICANAL: SOLO AGR        07 BICANAL: AGR & 1A1   634.5702 0.0033    151.066 1118.0744   True
04 UNICANAL: SOLO AGR 08 TRICANAL: ATM & AGR & 1A1   692.8519 0.0036   161.6496 1224.0543   True
05 UNICANAL: SOLO 1A1        06 BICANAL: ATM & 1A1        0.0    1.0  -384.5549  384.5549  False
05 UNICANAL: SOLO 1A1        07 BICANAL: AGR & 1A1  1787.2801    0.0  1477.8861 2096.6742   True
05 UNICANAL: SOLO 1A1 08 TRICANAL: ATM & AGR & 1A1  1845.5618    0.0  1465.9248 2225.1989   True
06 BICANAL: ATM & 1A1        0

### Corresponsales Uno a Uno

In [68]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import MultiComparison
#promedio_transacciones_atms_multifuncionales
modelo_anova = ols('conteo_transacciones_cbs_uno_a_uno ~ Cluster', data=df_cbs_1).fit()
anova_tabla = sm.stats.anova_lm(modelo_anova, typ=2)

# Crea un objeto MultiComparison para el análisis de Tukey
mc = MultiComparison(df_cbs_1['conteo_transacciones_cbs_uno_a_uno'], df_cbs_1['Cluster'])

# Realiza el análisis de Tukey
resultado_tukey = mc.tukeyhsd()

# Imprime el resultado del análisis de Tukey
print(resultado_tukey)

                      Multiple Comparison of Means - Tukey HSD, FWER=0.05                      
        group1                   group2             meandiff p-adj    lower      upper   reject
-----------------------------------------------------------------------------------------------
04 UNICANAL: SOLO AGR        05 UNICANAL: SOLO 1A1 2721.8773 0.0006   876.0409 4567.7138   True
04 UNICANAL: SOLO AGR        06 BICANAL: ATM & 1A1 2788.2385 0.0024    717.138  4859.339   True
04 UNICANAL: SOLO AGR        07 BICANAL: AGR & 1A1 5286.6682    0.0  3413.9671 7159.3692   True
04 UNICANAL: SOLO AGR 08 TRICANAL: ATM & AGR & 1A1 5197.9962    0.0  3140.5515  7255.441   True
05 UNICANAL: SOLO 1A1        06 BICANAL: ATM & 1A1   66.3612 0.9999 -1423.0909 1555.8132  False
05 UNICANAL: SOLO 1A1        07 BICANAL: AGR & 1A1 2564.7908    0.0  1366.4505 3763.1311   True
05 UNICANAL: SOLO 1A1 08 TRICANAL: ATM & AGR & 1A1 2476.1189 0.0001  1005.7145 3946.5233   True
06 BICANAL: ATM & 1A1        07 BICANAL:

# NOTA: 

Notemos que al realizar la prueba tuckey para los cluster por las variables "conteo_transacciones_cbs_uno_a_uno" y "conteo_transacciones_cbs_agregador" obtenemos los mismo resultados, es decir que estos canales se comportan de manera igual en los cluster analizados. Teniendo en cuenta que el portafolio transaccional es el mismo y que los establecimientos son del mismo tipo sin importar el formato de uno a uno o agregador, se opta por unir estas dos categorias en una sola, por comodidad lo nombramos como corresponsales informales.

In [60]:
df_cbs_1.groupby('Cluster')['conteo_transacciones_cbs_uno_a_uno'].sum()

Cluster
04 UNICANAL: SOLO AGR                0.000000
05 UNICANAL: SOLO 1A1           334790.913459
06 BICANAL: ATM & 1A1           147776.641148
07 BICANAL: AGR & 1A1           565673.493950
08 TRICANAL: ATM & AGR & 1A1    285889.793451
Name: conteo_transacciones_cbs_uno_a_uno, dtype: float64

In [70]:
df_cbs_1.groupby('Cluster')['conteo_transacciones_cbs_uno_a_uno'].mean()

Cluster
04 UNICANAL: SOLO AGR              0.000000
05 UNICANAL: SOLO 1A1           2721.877345
06 BICANAL: ATM & 1A1           2788.238512
07 BICANAL: AGR & 1A1           5286.668168
08 TRICANAL: ATM & AGR & 1A1    5197.996245
Name: conteo_transacciones_cbs_uno_a_uno, dtype: float64

In [62]:
df_cbs_1.groupby('Cluster')['conteo_transacciones_cbs_uno_a_uno'].median()

Cluster
04 UNICANAL: SOLO AGR              0.000000
05 UNICANAL: SOLO 1A1           1981.333333
06 BICANAL: ATM & 1A1           1519.285714
07 BICANAL: AGR & 1A1           4834.285714
08 TRICANAL: ATM & AGR & 1A1    4568.767857
Name: conteo_transacciones_cbs_uno_a_uno, dtype: float64

In [66]:
df_cbs_1.groupby('Cluster')['conteo_transacciones_cbs_agregador'].mean()

Cluster
04 UNICANAL: SOLO AGR           1152.709918
05 UNICANAL: SOLO 1A1              0.000000
06 BICANAL: ATM & 1A1              0.000000
07 BICANAL: AGR & 1A1           1787.280144
08 TRICANAL: ATM & AGR & 1A1    1845.561841
Name: conteo_transacciones_cbs_agregador, dtype: float64

### Corresponsales "Informales" (Uno a Uno - Agregadores)

In [75]:
df_cbs_1['conteo_transacciones_cbs_informal'] = df_cbs_1['conteo_transacciones_cbs_uno_a_uno'] + df_cbs_1['conteo_transacciones_cbs_agregador']

In [81]:
df_cbs_1.groupby('Cluster')['conteo_transacciones_cbs_informal'].mean()

Cluster
04 UNICANAL: SOLO AGR           1152.709918
05 UNICANAL: SOLO 1A1           2721.877345
06 BICANAL: ATM & 1A1           2788.238512
07 BICANAL: AGR & 1A1           7073.948312
08 TRICANAL: ATM & AGR & 1A1    7043.558086
Name: conteo_transacciones_cbs_informal, dtype: float64

In [82]:
df_cbs_1.groupby('Cluster')['conteo_transacciones_cbs_informal'].median()

Cluster
04 UNICANAL: SOLO AGR            862.357143
05 UNICANAL: SOLO 1A1           1981.333333
06 BICANAL: ATM & 1A1           1519.285714
07 BICANAL: AGR & 1A1           6600.976190
08 TRICANAL: ATM & AGR & 1A1    5797.325758
Name: conteo_transacciones_cbs_informal, dtype: float64

In [79]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import MultiComparison
#promedio_transacciones_atms_multifuncionales
modelo_anova = ols('conteo_transacciones_cbs_informal ~ Cluster', data=df_cbs_1).fit()
anova_tabla = sm.stats.anova_lm(modelo_anova, typ=2)

# Crea un objeto MultiComparison para el análisis de Tukey
mc = MultiComparison(df_cbs_1['conteo_transacciones_cbs_informal'], df_cbs_1['Cluster'])

# Realiza el análisis de Tukey
resultado_tukey = mc.tukeyhsd()

# Imprime el resultado del análisis de Tukey
print(resultado_tukey)

                      Multiple Comparison of Means - Tukey HSD, FWER=0.05                      
        group1                   group2             meandiff p-adj    lower      upper   reject
-----------------------------------------------------------------------------------------------
04 UNICANAL: SOLO AGR        05 UNICANAL: SOLO 1A1 1569.1674 0.2112  -454.1563 3592.4912  False
04 UNICANAL: SOLO AGR        06 BICANAL: ATM & 1A1 1635.5286 0.2803  -634.7195 3905.7767  False
04 UNICANAL: SOLO AGR        07 BICANAL: AGR & 1A1 5921.2384    0.0  3868.4669 7974.0099   True
04 UNICANAL: SOLO AGR 08 TRICANAL: ATM & AGR & 1A1 5890.8482    0.0  3635.5689 8146.1275   True
05 UNICANAL: SOLO 1A1        06 BICANAL: ATM & 1A1   66.3612    1.0 -1566.3099 1699.0322  False
05 UNICANAL: SOLO 1A1        07 BICANAL: AGR & 1A1  4352.071    0.0  3038.5037 5665.6382   True
05 UNICANAL: SOLO 1A1 08 TRICANAL: ATM & AGR & 1A1 4321.6807    0.0  2709.8889 5933.4725   True
06 BICANAL: ATM & 1A1        07 BICANAL:

## Conclusiones: 

1. A diferencia de las pruebas individuales, cuando se mezclan estos dos canales se arroja el resultado de que los cluster 4-5 y 4-6 se comportan de manera igual. Analizando este patron iniciando por los cluster 4-5 podemos ver que la diferencia es que en uno hay presencia de corresponsales agregadores (cluster 4) y en el otro corresponsales uno a uno (cluster 5), sin embargo, como ya fue mencionado a pesar de que esten individualmente el patron de comportamiento es el mismo; lo anterior se puede ver explicado por que no hay un cambio drastico en la manera de como el cliente percibe dicho establecimiento. Por otro lado si analizamos el cluster 4-6 percibimos un comportamiento similar donde individualmente hay presencia de corresponsales agregadores (cluster 4) y en el otro corresponsales uno a uno ademas de contar con que siempre hay presencia cajeros dispensadores (cluster 6). Teniendo en cuenta esta prueba y lo mencionado en las conclusiones con respecto a los ATMs dispensadores podemos corroborar de que la presencia de los ATMs solo muestra diferencias significativas (Mejor comportamiento) en lo transaccional cuando esta acompañando canales mas formales como lo son las sucursales o los corresponsales robustos.

2. Analizando los otros dos pares de cluster que segun la prueba se comportan de manera igual que son el 5-6 y 7-8, apoyandonos en lo mencionado en la conclusion anterior podemos corroborar dicho patron de comportamiento. en primer caso si observamos 5-6 la gran diferencia es que en el cluster 5 no hay presencia de atms dispensadores en cambio en el cluster 6 siempre hay. utilizando lo dicho en la conclusion anterior podemos decir que el hecho de tener atms que no esten aledaños a canales formales no esta representando un factor para impulsar el aumento del promedio de sus transacciones, contando solo con cbs en dichas zonas se podria decir que se estaria cumpliendo la cobertura de la zona. Para los cluster 7-8 pasa exactamente lo mismo

### Corresponsales "Formales" (Robustos)

In [89]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import MultiComparison
#promedio_transacciones_atms_multifuncionales
modelo_anova = ols('conteo_transacciones_cbs_robustos ~ Cluster', data=prueba).fit()
anova_tabla = sm.stats.anova_lm(modelo_anova, typ=2)

# Crea un objeto MultiComparison para el análisis de Tukey
mc = MultiComparison(prueba['conteo_transacciones_cbs_robustos'], prueba['Cluster'])

# Realiza el análisis de Tukey
resultado_tukey = mc.tukeyhsd()

# Imprime el resultado del análisis de Tukey
print(resultado_tukey)

C:\Users\jrecheve\AppData\Roaming\Python\Python39\site-packages\statsmodels\base\model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 9, but rank is 0
  warnings.warn('covariance of constraints does not have full '
C:\Users\jrecheve\AppData\Roaming\Python\Python39\site-packages\statsmodels\base\model.py:1917: RuntimeWarning: invalid value encountered in true_divide
  F /= J
C:\Users\jrecheve\AppData\Roaming\Python\Python39\site-packages\statsmodels\sandbox\stats\multicomp.py:1300: RuntimeWarning: invalid value encountered in true_divide
  st_range = np.abs(meandiffs) / std_pairs #studentized range statistic


                            Multiple Comparison of Means - Tukey HSD, FWER=0.05                            
               group1                               group2                meandiff p-adj lower upper reject
-----------------------------------------------------------------------------------------------------------
             00 MULTICANAL: Sucursal            01 MULTICANAL: CB Robusto      0.0   nan   0.0   0.0  False
             00 MULTICANAL: Sucursal 02 MULTICANAL: CB Robusto + Sucursal      0.0   nan   0.0   0.0  False
             00 MULTICANAL: Sucursal                03 UNICANAL: SOLO ATM      0.0   nan   0.0   0.0  False
             00 MULTICANAL: Sucursal                04 UNICANAL: SOLO AGR      0.0   nan   0.0   0.0  False
             00 MULTICANAL: Sucursal                05 UNICANAL: SOLO 1A1      0.0   nan   0.0   0.0  False
             00 MULTICANAL: Sucursal                06 BICANAL: ATM & 1A1      0.0   nan   0.0   0.0  False
             00 MULTICANAL: 

In [92]:
prueba.groupby('Cluster')['conteo_cbs_robustos'].mean()

Cluster
00 MULTICANAL: Sucursal                 0.000000
01 MULTICANAL: CB Robusto               1.000000
02 MULTICANAL: CB Robusto + Sucursal    1.000000
03 UNICANAL: SOLO ATM                   0.000000
04 UNICANAL: SOLO AGR                   0.000000
05 UNICANAL: SOLO 1A1                   0.000000
06 BICANAL: ATM & 1A1                   0.000000
07 BICANAL: AGR & 1A1                   0.000000
08 TRICANAL: ATM & AGR & 1A1            0.000000
99 De todo un poco (Noise)              0.736842
Name: conteo_cbs_robustos, dtype: float64

In [91]:
prueba.groupby('Cluster')['conteo_transacciones_cbs_robustos'].mean()

Cluster
00 MULTICANAL: Sucursal                 0.0
01 MULTICANAL: CB Robusto               0.0
02 MULTICANAL: CB Robusto + Sucursal    0.0
03 UNICANAL: SOLO ATM                   0.0
04 UNICANAL: SOLO AGR                   0.0
05 UNICANAL: SOLO 1A1                   0.0
06 BICANAL: ATM & 1A1                   0.0
07 BICANAL: AGR & 1A1                   0.0
08 TRICANAL: ATM & AGR & 1A1            0.0
99 De todo un poco (Noise)              0.0
Name: conteo_transacciones_cbs_robustos, dtype: float64

### Sucursales

In [116]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import MultiComparison
#promedio_transacciones_atms_multifuncionales
modelo_anova = ols('calificacion_promedio_sucursales ~ Cluster', data=df_suc).fit()
anova_tabla = sm.stats.anova_lm(modelo_anova, typ=2)

# Crea un objeto MultiComparison para el análisis de Tukey
mc = MultiComparison(df_suc['calificacion_promedio_sucursales'], df_suc['Cluster'])

# Realiza el análisis de Tukey
resultado_tukey = mc.tukeyhsd()

# Imprime el resultado del análisis de Tukey
print(resultado_tukey)

                       Multiple Comparison of Means - Tukey HSD, FWER=0.05                        
         group1                        group2                meandiff p-adj  lower   upper  reject
--------------------------------------------------------------------------------------------------
00 MULTICANAL: Sucursal 02 MULTICANAL: CB Robusto + Sucursal    6.658 0.0005 3.0221 10.2938   True
--------------------------------------------------------------------------------------------------


In [118]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import MultiComparison
#promedio_transacciones_atms_multifuncionales
modelo_anova = ols('suma_tc ~ Cluster', data=df_suc).fit()
anova_tabla = sm.stats.anova_lm(modelo_anova, typ=2)

# Crea un objeto MultiComparison para el análisis de Tukey
mc = MultiComparison(df_suc['suma_tc'], df_suc['Cluster'])

# Realiza el análisis de Tukey
resultado_tukey = mc.tukeyhsd()

# Imprime el resultado del análisis de Tukey
print(resultado_tukey)

                             Multiple Comparison of Means - Tukey HSD, FWER=0.05                             
         group1                        group2                 meandiff  p-adj     lower       upper    reject
-------------------------------------------------------------------------------------------------------------
00 MULTICANAL: Sucursal 02 MULTICANAL: CB Robusto + Sucursal 53580.7216 0.3638 -63133.0012 170294.4444  False
-------------------------------------------------------------------------------------------------------------


In [119]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import MultiComparison

modelo_anova = ols('conteo_clientes ~ Cluster', data=df_suc).fit()
anova_tabla = sm.stats.anova_lm(modelo_anova, typ=2)

# Crea un objeto MultiComparison para el análisis de Tukey
mc = MultiComparison(df_suc['conteo_clientes'], df_suc['Cluster'])

# Realiza el análisis de Tukey
resultado_tukey = mc.tukeyhsd()

# Imprime el resultado del análisis de Tukey
print(resultado_tukey)

                          Multiple Comparison of Means - Tukey HSD, FWER=0.05                          
         group1                        group2                 meandiff p-adj   lower     upper   reject
-------------------------------------------------------------------------------------------------------
00 MULTICANAL: Sucursal 02 MULTICANAL: CB Robusto + Sucursal 7098.0561 0.0244 938.4961 13257.616   True
-------------------------------------------------------------------------------------------------------


# Nota: 
Santi para corresponsales robustos me di cuenta que no contamos con transacciones desde la primera base que nos habia enviado Santiago Valencia, Seria entonces analizar este canal con los diferentes indicadores para dicho formato para ver que podemos encontrar. y en cuanto sucursales de las unicas variable que incluimos dentro de la base es la calificación de la sucursal y la suma del tamaño comercial. por la pruebase evidencia que las calificaciones promedio son diferentes en los dos cluster analizados que son el 0 y el 2, y por la diferencia de medias se ve que el que tiene mayor calificacion promedio es cluster 2. en cuanto a suma de tamaño comercial se comporta igual ambos clusters, y por el ultimo al revisar el conteo de cliente en estos cluster sucede al igual que en la suma del tamaño comercial el comportamiento es diferente y el conteo de clientes es mayor en cluster 2 